In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(40)

# Data set
n_samples = 20
s1                   = np.logspace(-1, 1, n_samples) 
s2                   = np.logspace(-1, 1, n_samples) 

data_s1, data_s2 =  np.meshgrid(s1, s2)
data = np.array([data_s1.ravel(), data_s2.ravel()]).T
n_70          = int(0.7 * len(data))


# Mandatory training set
mandatory_train   = {0, 19, 380, 399} # corners
mandatory_test    = {13, 332, 149}  # ploted points
remaining_indices = set(range(0, len(data))) - mandatory_train - mandatory_test
n_mandatory_train = len(mandatory_train)

# Generate random train/test split
train_indices     = set(np.random.choice(list(remaining_indices), size= n_70-n_mandatory_train, replace=False))
train_indices    |= mandatory_train
test_indices      = set(range(len(data))) - train_indices

train_indices = np.array(sorted(train_indices))
test_indices  = np.array(sorted(test_indices))

# Selection of 7 random indices and the mandatory test indices
test_subset       = np.array(test_indices[np.random.choice(len(test_indices), 10, replace=False)])
alpha_test_subset = data[test_subset]
alpha_special     = data[list(mandatory_test)]

# Create DataFrames
dataset_df = pd.DataFrame(data, columns=['s1', 's2'])
indices_train_df = pd.DataFrame({'train_indices': train_indices})
indices_test_df  = pd.DataFrame({'test_indices': test_indices})
set_test_subset_df = pd.DataFrame(test_subset)
set_special_df = pd.DataFrame(list(mandatory_test))



# Save to CSV files
folder = 'Dataset/'
dataset_df.to_csv(folder + 'dataset.csv', index=False)
indices_train_df.to_csv(folder + 'train_indices.csv', index=False)
indices_test_df.to_csv(folder + 'test_indices.csv', index=False)
set_test_subset_df.to_csv(folder + 'test_subset.csv', index=False)
set_special_df.to_csv(folder + 'special_test.csv', index=False)


In [4]:
_train_data = np.load('ngsolve/train_data.npz')
sigma_ritz_train = _train_data['sigma']
ritz_train = _train_data['Ritz']

_val_data = np.load('ngsolve/val_data.npz')
sigma_ritz_val = _val_data['sigma']
ritz_val = _val_data['Ritz']

sigma_all = np.concatenate((sigma_ritz_train, sigma_ritz_val))


In [5]:
ritz_all = np.concatenate((ritz_train, ritz_val))

In [6]:
ritz_all_order = []
sigma_all_order = []

for i in range(len(ritz_all)):
    for j in range(len(ritz_all)):
        if np.all(np.isclose(sigma_all[j], data[i])):
            ritz_all_order.append(ritz_all[j])
            sigma_all_order.append(sigma_all[j])
            break


In [7]:
ritz_all_order = np.array(ritz_all_order)
sigma_all_order = np.array(sigma_all_order)

In [8]:
np.save('ngsolve/ritz_exact_all.npy', ritz_all_order)
np.save('ngsolve/sigma_all.npy', sigma_all_order)